In [1]:
import ifcopenshell
import numpy as np
import pandas as pd

In [4]:
model_path = r"C:\Users\relia\Documents\GitHub\DigitalTwins\lattice_tower.ifc"
ifc_model = ifcopenshell.open(model_path)

# Display some information about the model
ifc_model

In [31]:
# Retrieve all instances of IfcBeam
beams = ifc_model.by_type("IfcBeam")

# Select the first beam
first_beam = beams[0]

# Print basic properties
print(f"ID: {first_beam.id()}")
print(f"Name: {first_beam.Name}")
print(f"Description: {first_beam.Description}")
print(f"ObjectType: {first_beam.ObjectType}")
print(f"GlobalId: {first_beam.GlobalId}")
print(f"Type: {first_beam.is_a()}")

# Explore its representation
if hasattr(first_beam, 'Representation') and first_beam.Representation:
    print("\nRepresentations:")
    for rep in first_beam.Representation.Representations:
        print(f"  Representation Type: {rep.RepresentationType}")
        for item in rep.Items:
            print(f"    Item Type: {item.is_a()}")
            if item.is_a('IfcExtrudedAreaSolid'):
                profile = item.SweptArea
                print(f"      Profile Type: {profile.is_a()}")
                # Additional attributes based on profile type
                if profile.is_a("IfcRectangleProfileDef"):
                    print(f"        XDim: {profile.XDim}")
                    print(f"        YDim: {profile.YDim}")
                elif profile.is_a("IfcCircleProfileDef"):
                    print(f"        Radius: {profile.Radius}")
                elif profile.is_a("IfcLShapeProfileDef"):
                    print(f"        Depth: {profile.Depth}")
                    print(f"        Width: {profile.Width}")
                    print(f"        Thickness: {profile.Thickness}")
                    print(f"        FilletRadius: {profile.FilletRadius}")
                elif profile.is_a("IfcArbitraryClosedProfileDef"):
                    print(f"        OuterCurve: {profile.OuterCurve.is_a()}")
                    # Add logic to explore the outer curve if needed

# Check associated relationships (e.g., materials)
material = None
material_relationships = ifc_model.get_inverse(first_beam, "HasAssociations")
for rel in material_relationships:
    if rel.is_a("IfcRelAssociatesMaterial"):
        material = rel.RelatingMaterial.Name
        print(f"\nMaterial: {material}")

# Add any additional checks or explorations as needed

ID: 258
Name: L_55_beam:L_55_beam:426998
Description: None
ObjectType: L_55_beam:L_55_beam
GlobalId: 3eYuxTHWbCu8VwKAEE98fJ
Type: IfcBeam

Representations:
  Representation Type: Curve2D
    Item Type: IfcPolyline
  Representation Type: SweptSolid
    Item Type: IfcExtrudedAreaSolid
      Profile Type: IfcArbitraryClosedProfileDef
        OuterCurve: IfcCompositeCurve

Material: Steel - Red


In [33]:
import ifcopenshell

# Open the IFC model
model_path = r"C:\Users\relia\Documents\GitHub\DigitalTwins\lattice_tower.ifc"
ifc_model = ifcopenshell.open(model_path)

# Retrieve all instances of IfcBeam
beams = ifc_model.by_type("IfcBeam")

# Select the first beam
first_beam = beams[0]

# Print basic properties
print(f"ID: {first_beam.id()}")
print(f"Name: {first_beam.Name}")
print(f"Description: {first_beam.Description}")
print(f"ObjectType: {first_beam.ObjectType}")
print(f"GlobalId: {first_beam.GlobalId}")
print(f"Type: {first_beam.is_a()}")

# Print all attributes of the beam, including inherited ones
print("\nAll attributes of the beam (including inherited):")
for attr_name in dir(first_beam):
    if not attr_name.startswith("__") and not callable(getattr(first_beam, attr_name)):
        print(f"{attr_name}: {getattr(first_beam, attr_name)}")

# Print all relationships
print("\nAll relationships of the beam:")
for rel in ifc_model.get_inverse(first_beam):
    print(f"Relationship Type: {rel.is_a()}")
    for related in rel.__dict__.values():
        if isinstance(related, list):
            for item in related:
                print(f"  Related Item: {item}")
        else:
            print(f"  Related Item: {related}")

ID: 258
Name: L_55_beam:L_55_beam:426998
Description: None
ObjectType: L_55_beam:L_55_beam
GlobalId: 3eYuxTHWbCu8VwKAEE98fJ
Type: IfcBeam

All attributes of the beam (including inherited):
ConnectedFrom: ()
ConnectedTo: ()
ContainedInStructure: (#59685=IfcRelContainedInSpatialStructure('1w2kgyXAD2C9c7QsP4npO9',#41,$,$,(#258,#416,#533,#667,#784,#907,#1034,#1151,#1268,#1385,#1502,#1619,#1736,#1851,#1966,#2081,#2196,#2311,#2426,#2541,#2656,#2771,#2886,#3001,#3116,#3231,#3346,#3461,#3576,#5601,#8513,#8629,#8746,#8863,#8980,#9097,#9214,#9331,#9448,#9565,#9682,#9799,#9916,#10033,#10148,#10263,#10378,#10493,#10608,#10723,#10838,#10953,#11068,#11183,#11298,#11413,#11528,#11643,#11758,#11873,#13997,#16971,#17087,#17204,#17321,#17438,#17555,#17672,#17789,#17906,#18023,#18140,#18257,#18372,#18487,#18602,#18717,#18832,#18947,#19062,#19177,#19292,#19407,#19522,#19637,#19752,#19867,#19982,#20097,#22249,#25194,#25311,#25428,#25545,#25662,#25779,#25896,#26013,#26130,#26247,#26364,#26481,#26596,#26711,

In [27]:
# Retrieve all instances of IfcBeam
beams = ifc_model.by_type("IfcBeam")

# Helper function to get material name


def get_material(beam):
    material = None
    material_relationships = ifc_model.get_inverse(beam, "HasAssociations")
    for rel in material_relationships:
        if rel.is_a("IfcRelAssociatesMaterial"):
            material = rel.RelatingMaterial.Name
    return material


# Collect the data for each beam
beam_data = []
for beam in beams:
    # Get material
    material = get_material(beam)

    # Create beam info dictionary
    beam_info = {
        "ID": beam.id(),
        "Name": beam.Name,
        "ObjectType": beam.ObjectType,
        "Type": beam.is_a(),
        "Material": material
    }

    # Get geometric data (example: span)
    if hasattr(beam, 'Representation') and beam.Representation:
        for rep in beam.Representation.Representations:
            if rep.RepresentationType == 'Curve2D':  # This is just an example, adjust as needed
                for item in rep.Items:
                    if item.is_a('IfcPolyline'):
                        points = item.Points
                        if len(points) >= 2:
                            span = points[1].Coordinates[0] - \
                                points[0].Coordinates[0]
                            beam_info["Span"] = span
                            break

    beam_data.append(beam_info)

# Create a DataFrame
beam_df = pd.DataFrame(beam_data)

# Display the DataFrame
beam_df

,ID,Name,ObjectType,Type,Material,Span
0,258,L_55_beam:L_55_beam:426998,L_55_beam:L_55_beam,IfcBeam,Steel - Red,2147.364468
1,416,L_55_beam:L_55_beam:431618,L_55_beam:L_55_beam,IfcBeam,Steel - Red,2147.219037
2,533,L_35_beam:L_35_beam:432557,L_35_beam:L_35_beam,IfcBeam,Steel - Red,770.087291
3,667,L_35_beam:L_35_beam:432574,L_35_beam:L_35_beam,IfcBeam,Steel - Red,784.432632
4,784,L_35_beam:L_35_beam:432589,L_35_beam:L_35_beam,IfcBeam,Steel - Red,799.523595
...,...,...,...,...,...,...
147,59168,L_55_beam:L_55_beam:456762,L_55_beam:L_55_beam,IfcBeam,Steel - White,2681.437434
148,59285,L_55_beam:L_55_beam:457153,L_55_beam:L_55_beam,IfcBeam,Steel - Red,7479.478553
149,59402,L_55_beam:L_55_beam:457400,L_55_beam:L_55_beam,IfcBeam,Steel - Red,7478.930978
150,59519,L_55_beam:L_55_beam:457519,L_55_beam:L_55_beam,IfcBeam,Steel - Red,7478.120045


In [18]:
beam_df["ObjectType"].value_counts()

ObjectType
L_35_beam:L_35_beam    132
L_55_beam:L_55_beam     20
Name: count, dtype: int64

In [35]:
import ifcopenshell

# Load the IFC file (assuming it's already loaded into the variable `ifc_file`)
beam = ifc_model.by_id(258)

# Get the profile definition
extruded_area_solid = next(
    rep for rep in beam.Representation.Representations if rep.RepresentationType == 'SweptSolid')
profile_def = extruded_area_solid.Items[0].SweptArea

# Check the type to ensure it is IfcArbitraryClosedProfileDef
if profile_def.is_a('IfcArbitraryClosedProfileDef'):
    outer_curve = profile_def.OuterCurve

    # Explore the segments of IfcCompositeCurve
    if outer_curve.is_a('IfcCompositeCurve'):
        segments = outer_curve.Segments

        segment_details = []

        for segment in segments:
            curve = segment.ParentCurve
            if curve.is_a('IfcPolyline'):
                points = [(point.Coordinates[0], point.Coordinates[1])
                          for point in curve.Points]
                segment_details.append(
                    {'type': 'IfcPolyline', 'points': points})
            elif curve.is_a('IfcTrimmedCurve'):
                base_curve = curve.BasisCurve
                # Handle IfcTrimmedCurve with specific details
                # This example assumes handling IfcLine and IfcCircle as base curves
                if base_curve.is_a('IfcLine'):
                    start_point = base_curve.Pnt.Coordinates
                    direction = base_curve.DirVector.DirectionRatios
                    segment_details.append(
                        {'type': 'IfcLine', 'start_point': start_point, 'direction': direction})
                elif base_curve.is_a('IfcCircle'):
                    radius = base_curve.Radius
                    center = base_curve.Position.Location.Coordinates
                    segment_details.append(
                        {'type': 'IfcCircle', 'center': center, 'radius': radius})
            else:
                segment_details.append(
                    {'type': curve.is_a(), 'details': 'Other curve type'})

# Print segment details
for detail in segment_details:
    print(detail)

{'type': 'IfcPolyline', 'points': [(-36.4285714285714, -18.5714285714297), (18.5714285714286, -18.5714285714295)]}
{'type': 'IfcPolyline', 'points': [(18.5714285714287, -18.5714285714295), (18.5714285714287, 36.4285714285705)]}
{'type': 'IfcCircle', 'center': (18.5714285714289, 28.4285714285705), 'radius': 8.00000000000005}
{'type': 'IfcPolyline', 'points': [(10.5714285714288, 28.4285714285705), (10.5714285714288, -6.57142857142945)]}
{'type': 'IfcCircle', 'center': (6.57142857142901, -6.57142857142958), 'radius': 3.99999999999976}
{'type': 'IfcPolyline', 'points': [(6.5714285714286, -10.5714285714296), (-28.4285714285712, -10.5714285714296)]}
{'type': 'IfcCircle', 'center': (-28.4285714285713, -18.5714285714296), 'radius': 8.00000000000005}


In [36]:
import ifcopenshell

# Load your IFC model (assuming you have already done this)
# ifc_model = ifcopenshell.open('path_to_your_ifc_file.ifc')

# Retrieve the beam object by its ID
beam = ifc_model.by_id(258)

# Access the Representation attribute of the beam
representation = beam.Representation

# Find the SweptSolid representation
for rep in representation.Representations:
    if rep.RepresentationType == 'SweptSolid':
        # Access the IfcExtrudedAreaSolid
        for item in rep.Items:
            if item.is_a('IfcExtrudedAreaSolid'):
                # Access the profile definition
                profile_def = item.SweptArea

                # Retrieve the profile name if it exists
                profile_name = profile_def.ProfileName if hasattr(
                    profile_def, 'ProfileName') else None

                print(f"Profile Name: {profile_name}")
                break

Profile Name: L_55_beam


In [45]:
beam = ifc_model.by_id(258)
print(f"Retrieved beam: {beam}")

Retrieved beam: #258=IfcBeam('3eYuxTHWbCu8VwKAEE98fJ',#41,'L_55_beam:L_55_beam:426998',$,'L_55_beam:L_55_beam',#164,#254,'426998')


In [46]:
print("Checking IsDefinedBy relationships...")
beam_type = None
for rel in beam.IsDefinedBy:
    print(f"Found relationship: {rel}")
    if rel.is_a('IfcRelDefinesByType'):
        beam_type = rel.RelatingType
        print(f"Found beam type: {beam_type}")
        break

if not beam_type:
    print("No IfcRelDefinesByType relationship found.")

Checking IsDefinedBy relationships...
Found relationship: #297=IfcRelDefinesByProperties('2GuQVpZUT4Xx5ZHWECL23E',#41,$,$,(#258),#287)
Found relationship: #301=IfcRelDefinesByProperties('0ajaCixkPFmhrJ6lNC8wL5',#41,$,$,(#258),#290)
Found relationship: #304=IfcRelDefinesByProperties('15q_fGMiXFouYzS2bFr1_R',#41,$,$,(#258),#292)
Found relationship: #307=IfcRelDefinesByProperties('2Y4Pe8XkP67f2QCOqWUujg',#41,$,$,(#258),#295)
Found relationship: #60026=IfcRelDefinesByType('0DP9pyX$PDx9c8UMKSJBhT',#41,$,$,(#258,#416,#8629,#8746,#57881,#57998,#58115,#58232,#58349,#58466,#58583,#58700,#58817,#58934,#59051,#59168,#59285,#59402,#59519,#59636),#267)
Found beam type: #267=IfcBeamType('0H8HAJpSz2n8nptd5vnEy$',#41,'L_55_beam:L_55_beam',$,$,(#312,#315,#317,#320),$,'431447',$,.NOTDEFINED.)


In [47]:
print("Retrieving geometric representation...")
representation = beam.Representation.Representations
swept_solid = None
for rep in representation:
    print(f"Checking representation: {rep}")
    if rep.RepresentationType == 'SweptSolid':
        swept_solid = rep
        print(f"Found SweptSolid representation: {swept_solid}")
        break

if not swept_solid:
    print("No SweptSolid representation found.")

Retrieving geometric representation...
Checking representation: #251=IfcShapeRepresentation(#115,'Axis','Curve2D',(#249))
Checking representation: #244=IfcShapeRepresentation(#117,'Body','SweptSolid',(#234))
Found SweptSolid representation: #244=IfcShapeRepresentation(#117,'Body','SweptSolid',(#234))


In [48]:
if swept_solid:
    profile = swept_solid.Items[0].SweptArea
    profile_name = profile.ProfileName
    print(f"Retrieved profile: {profile}")
    print(f"Profile name: {profile_name}")
else:
    print("Cannot retrieve profile as no SweptSolid representation was found.")

Retrieved profile: #228=IfcArbitraryClosedProfileDef(.AREA.,'L_55_beam',#219)
Profile name: L_55_beam


In [49]:
# Step 1: Retrieve the Beam
beam = ifc_model.by_id(258)
print(f"Retrieved beam: {beam}")

# Step 2: Identify Relationships
print("Checking IsDefinedBy relationships...")
beam_type = None
for rel in beam.IsDefinedBy:
    print(f"Found relationship: {rel}")
    if rel.is_a('IfcRelDefinesByType'):
        beam_type = rel.RelatingType
        print(f"Found beam type: {beam_type}")
        break

if not beam_type:
    print("No IfcRelDefinesByType relationship found.")

# Step 3: Get the Representation
print("Retrieving geometric representation...")
representation = beam.Representation.Representations
swept_solid = None
for rep in representation:
    print(f"Checking representation: {rep}")
    if rep.RepresentationType == 'SweptSolid':
        swept_solid = rep
        print(f"Found SweptSolid representation: {swept_solid}")
        break

if not swept_solid:
    print("No SweptSolid representation found.")

# Step 4: Retrieve the Profile
if swept_solid:
    profile = swept_solid.Items[0].SweptArea
    profile_name = profile.ProfileName
    print(f"Retrieved profile: {profile}")
    print(f"Profile name: {profile_name}")
else:
    print("Cannot retrieve profile as no SweptSolid representation was found.")

Retrieved beam: #258=IfcBeam('3eYuxTHWbCu8VwKAEE98fJ',#41,'L_55_beam:L_55_beam:426998',$,'L_55_beam:L_55_beam',#164,#254,'426998')
Checking IsDefinedBy relationships...
Found relationship: #297=IfcRelDefinesByProperties('2GuQVpZUT4Xx5ZHWECL23E',#41,$,$,(#258),#287)
Found relationship: #301=IfcRelDefinesByProperties('0ajaCixkPFmhrJ6lNC8wL5',#41,$,$,(#258),#290)
Found relationship: #304=IfcRelDefinesByProperties('15q_fGMiXFouYzS2bFr1_R',#41,$,$,(#258),#292)
Found relationship: #307=IfcRelDefinesByProperties('2Y4Pe8XkP67f2QCOqWUujg',#41,$,$,(#258),#295)
Found relationship: #60026=IfcRelDefinesByType('0DP9pyX$PDx9c8UMKSJBhT',#41,$,$,(#258,#416,#8629,#8746,#57881,#57998,#58115,#58232,#58349,#58466,#58583,#58700,#58817,#58934,#59051,#59168,#59285,#59402,#59519,#59636),#267)
Found beam type: #267=IfcBeamType('0H8HAJpSz2n8nptd5vnEy$',#41,'L_55_beam:L_55_beam',$,$,(#312,#315,#317,#320),$,'431447',$,.NOTDEFINED.)
Retrieving geometric representation...
Checking representation: #251=IfcShapeRepres

In [52]:
# Step 1: Retrieve the Beam
beam = ifc_model.by_id(258)
print(f"Retrieved beam: {beam}")

# Step 2: Identify Relationships
print("Checking IsDefinedBy relationships...")
beam_type = None
for rel in beam.IsDefinedBy:
    print(f"Found relationship: {rel}")
    if rel.is_a('IfcRelDefinesByType'):
        beam_type = rel.RelatingType
        print(f"Found beam type: {beam_type}")
        break

if not beam_type:
    print("No IfcRelDefinesByType relationship found.")

# Step 3: Get the Representation
print("Retrieving geometric representation...")
representation = beam.Representation.Representations
swept_solid = None
for rep in representation:
    print(f"Checking representation: {rep}")
    if rep.RepresentationType == 'SweptSolid':
        swept_solid = rep
        print(f"Found SweptSolid representation: {swept_solid}")
        break

if not swept_solid:
    print("No SweptSolid representation found.")

# Step 4: Retrieve the Profile
if swept_solid:
    profile = swept_solid.Items[0].SweptArea
    profile_name = profile.ProfileName
    profile_type = profile.ProfileType
    print(f"Retrieved profile: {profile}")
    print(f"Profile name: {profile_name}")
    print(f"Profile type: {profile_type}")
else:
    print("Cannot retrieve profile as no SweptSolid representation was found.")

# Step 5: Verify Profile Name
if profile_name != 'L_55_profile':
    print(f"Profile name mismatch: expected 'L_55_profile', got '{
          profile_name}'")
else:
    print(f"Profile name is correct: {profile_name}")

# Output the Profile Type
print(f"Profile type (expected .AREA.): {profile_type}")

Retrieved beam: #258=IfcBeam('3eYuxTHWbCu8VwKAEE98fJ',#41,'L_55_beam:L_55_beam:426998',$,'L_55_beam:L_55_beam',#164,#254,'426998')
Checking IsDefinedBy relationships...
Found relationship: #297=IfcRelDefinesByProperties('2GuQVpZUT4Xx5ZHWECL23E',#41,$,$,(#258),#287)
Found relationship: #301=IfcRelDefinesByProperties('0ajaCixkPFmhrJ6lNC8wL5',#41,$,$,(#258),#290)
Found relationship: #304=IfcRelDefinesByProperties('15q_fGMiXFouYzS2bFr1_R',#41,$,$,(#258),#292)
Found relationship: #307=IfcRelDefinesByProperties('2Y4Pe8XkP67f2QCOqWUujg',#41,$,$,(#258),#295)
Found relationship: #60026=IfcRelDefinesByType('0DP9pyX$PDx9c8UMKSJBhT',#41,$,$,(#258,#416,#8629,#8746,#57881,#57998,#58115,#58232,#58349,#58466,#58583,#58700,#58817,#58934,#59051,#59168,#59285,#59402,#59519,#59636),#267)
Found beam type: #267=IfcBeamType('0H8HAJpSz2n8nptd5vnEy$',#41,'L_55_beam:L_55_beam',$,$,(#312,#315,#317,#320),$,'431447',$,.NOTDEFINED.)
Retrieving geometric representation...
Checking representation: #251=IfcShapeRepres